# Prediction of sales

### Problem Statement
[The dataset](https://drive.google.com/file/d/1B07fvYosBNdIwlZxSmxDfeAf9KaygX89/view?usp=sharing) represents sales data for 1559 products across 10 stores in different cities. Also, attributes of each product and store are available. The aim is to build a predictive model and determine the sales of each product at a particular store.

|Variable|Description|
|: ------------- |:-------------|
|Item_Identifier|Unique product ID|
|Item_Weight|Weight of product|
|Item_Fat_Content|Whether the product is low fat or not|
|Item_Visibility|The % of total display area of all products in a store allocated to the particular product|
|Item_Type|The category to which the product belongs|
|Item_MRP|Maximum Retail Price (list price) of the product|
|Outlet_Identifier|Unique store ID|
|Outlet_Establishment_Year|The year in which store was established|
|Outlet_Size|The size of the store in terms of ground area covered|
|Outlet_Location_Type|The type of city in which the store is located|
|Outlet_Type|Whether the outlet is just a grocery store or some sort of supermarket|
|Item_Outlet_Sales|Sales of the product in the particulat store. This is the outcome variable to be predicted.|

Please note that the data may have missing values as some stores might not report all the data due to technical glitches. Hence, it will be required to treat them accordingly.



### In following weeks, we will explore the problem in following stages:

1. **Hypothesis Generation – understanding the problem better by brainstorming possible factors that can impact the outcome**
2. **Data Exploration – looking at categorical & continuous feature summaries and making inferences about the data**
3. **Data Cleaning – imputing missing values in the data and checking for outliers**
4. **Feature Engineering – modifying existing variables and/or creating new ones for analysis**
5. **Model Building – making predictive models on the data**
---------

In [1]:
import pandas as pd
import numpy as np

#Read files:
data = pd.read_csv("cleaned.csv", delimiter=',')

## 4. Feature Engineering

1. Resolving the issues in the data to make it ready for the analysis.
2. Create some new variables using the existing ones.





### Create a broad category of Type of Item

`Item_Type` variable has many categories which might prove to be very useful in analysis. Look at the `Item_Identifier`, i.e. the unique ID of each item, it starts with either FD, DR or NC. If you see the categories, these look like being Food, Drinks and Non-Consumables. 

**Task:** Use the Item_Identifier variable to create a new column

In [2]:
data['Item_Identifier_Type'] = data.Item_Identifier.astype(str).str[:2]

### Determine the years of operation of a store

**Task:** Make a new column depicting the years of operation of a store (i.e. how long the store exists). 

In [3]:
data['current_year'] = 2021
data['Operation_Year'] = data.current_year - data.Outlet_Establishment_Year

In [4]:
data = data.drop('current_year', axis=1)

### Modify categories of Item_Fat_Content

**Task:** There are difference in representation in categories of Item_Fat_Content variable. This should be corrected.

In [5]:
data.loc[data['Item_Fat_Content'] == 'LF', 'Item_Fat_Content'] = 'Low Fat'


In [6]:
data.loc[data['Item_Fat_Content'] == 'low fat', 'Item_Fat_Content'] = 'Low Fat'
data.loc[data['Item_Fat_Content'] == 'reg', 'Item_Fat_Content'] = 'Regular'

**Task:** There are some non-consumables as well and a fat-content should not be specified for them. Create a separate category for such kind of observations.

In [7]:
data.Item_Type.unique()

array(['Dairy', 'Soft Drinks', 'Meat', 'Fruits and Vegetables',
       'Household', 'Baking Goods', 'Snack Foods', 'Frozen Foods',
       'Breakfast', 'Health and Hygiene', 'Hard Drinks', 'Canned',
       'Breads', 'Starchy Foods', 'Others', 'Seafood'], dtype=object)

In [8]:
data = data.replace({
    'Outlet_Location_Type': {'Tier 1': 1, 'Tier 2': 2, 'Tier 3': 3}
    
})

In [9]:
data['Wrong_fat_content'] = data['Item_Type'].apply(lambda x: 1 if x in ['Household','Health and Hygiene'] else 0)

In [10]:
data.loc[data['Item_Identifier_Type'] == 'NC', 'Item_Fat_Content'] = 'None'

In [11]:
data.loc[data['Item_Type'] == 'Household', 'Item_Fat_Content'] = 'None'

In [12]:
data.loc[data['Item_Type'] == 'Health and Hygiene', 'Item_Fat_Content'] = 'None'

In [13]:
data = data.drop('Wrong_fat_content', axis = 1)

In [14]:
data2 = data

### Numerical and One-Hot Encoding of Categorical variables

Since scikit-learn algorithms accept only numerical variables, we need to **convert all categorical variables into numeric types.** 

- if the variable is Ordinal we can simply map its values into numbers
- if the variable is Nominal (we cannot sort the values) we need to One-Hot Encode them --> create dummy variables

In [15]:
data_type = data[['Item_Identifier', 'Item_Type']]

In [16]:
# generate binary values using get_dummies
dum_df = pd.get_dummies(data_type, columns=["Item_Type"], prefix=["Type_is"] )
dum_df
# merge with main df bridge_df on key values
# data_type = data_type.join(dum_df)
# data_type

,Item_Identifier,Type_is_Baking Goods,Type_is_Breads,Type_is_Breakfast,Type_is_Canned,Type_is_Dairy,Type_is_Frozen Foods,Type_is_Fruits and Vegetables,Type_is_Hard Drinks,Type_is_Health and Hygiene,Type_is_Household,Type_is_Meat,Type_is_Others,Type_is_Seafood,Type_is_Snack Foods,Type_is_Soft Drinks,Type_is_Starchy Foods
0,FDA15,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,DRC01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,FDN15,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,FDX07,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,NCD19,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8519,FDS36,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8520,NCJ29,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
8521,FDN46,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [17]:
data_outlet_size  = data [['Item_Identifier', 'Outlet_Size']]
dum_df2 = pd.get_dummies(data_outlet_size, columns=["Outlet_Size"], prefix=["Type_is"] )

In [18]:
dum_df2

,Item_Identifier,Type_is_High,Type_is_Medium,Type_is_Small
0,FDA15,0,1,0
1,DRC01,0,1,0
2,FDN15,0,1,0
3,FDX07,0,1,0
4,NCD19,1,0,0
...,...,...,...,...
8518,FDF22,1,0,0
8519,FDS36,0,1,0
8520,NCJ29,0,0,1
8521,FDN46,0,1,0


**All variables should be by now numeric.**

In [19]:
data_fat_content  = data [['Item_Identifier', 'Item_Fat_Content']]
dum_df3 = pd.get_dummies(data_fat_content, columns=["Item_Fat_Content"], prefix=["Type_is"] )

In [20]:
data_fat_content  = data [['Item_Identifier', 'Item_Fat_Content']]
dum_df3 = pd.get_dummies(data_fat_content, columns=["Item_Fat_Content"], prefix=["Type_is"] )

---------
### Exporting Data

**Task:** You can save the processed data to your local machine as a csv file.

In [21]:
#Import library:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#New variable for outlet
data2['Outlet'] = le.fit_transform(data2['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Identifier_Type','Outlet_Type','Outlet','Item_Type']
le = LabelEncoder()
for i in var_mod:
    data2[i] = le.fit_transform(data2[i])

In [23]:
data2.to_csv('testdata.csv')